In [1]:
import os
import numpy as np
import pandas as pd
import random
import time
import scanpy as sc
import torch

import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO)
import scDualGN

def seed_torch(seed=666):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
seed_torch()

In [2]:
#load data
adata = sc.read_h5ad('/media/bgi/zhouruilong/sc/Human/Pancreas/baron_human.h5ad')
adata.obs['celltype'] = adata.obs['cell_type1']

In [3]:
sc.pp.filter_cells(adata, min_counts=200)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata,target_sum=1e6)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2500)
adata = adata[:, adata.var.highly_variable]
#sc.pp.scale(adata, max_value=10)
sc.pp.scale(adata)

In [4]:
#kmeans
start = time.time()
z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata, n_cluster=14,n_z=64,n_epoch_update_pq=5,alpha_dualvae=0.01, beta_daulvae=20, gamma_dualvae=2,
                                                   n_epochs=30,GPU_id=1)
end = time.time()

print('running time = {}'.format(end-start))

INFO:scDualGN.run:device: cuda:1


0.001


INFO:scDualGN.run:pretrain device: cuda:1
INFO:scDualGN.run:start pretraining...
INFO:scDualGN.run:Epoch 0/100,Overall loss:49539.5726,MSE:2460.7269,MSE1:162.2445,KL: 54.4075
INFO:scDualGN.run:Epoch 1/100,Overall loss:44019.9979,MSE:2199.3656,MSE1:15.7592,KL: 116.6101
INFO:scDualGN.run:Epoch 2/100,Overall loss:43080.2564,MSE:2153.1275,MSE1:8.0691,KL: 156.8159
INFO:scDualGN.run:Epoch 3/100,Overall loss:42780.6406,MSE:2138.3810,MSE1:5.6029,KL: 181.5329
INFO:scDualGN.run:Epoch 4/100,Overall loss:42373.4442,MSE:2118.0901,MSE1:4.8801,KL: 188.2518
INFO:scDualGN.run:Epoch 5/100,Overall loss:41983.2466,MSE:2098.5922,MSE1:4.7193,KL: 196.3730
INFO:scDualGN.run:Epoch 6/100,Overall loss:41646.1264,MSE:2081.8089,MSE1:3.9140,KL: 212.0777
INFO:scDualGN.run:Epoch 7/100,Overall loss:41418.1126,MSE:2070.4551,MSE1:3.3777,KL: 225.4695
INFO:scDualGN.run:Epoch 8/100,Overall loss:41150.3143,MSE:2057.0961,MSE1:3.0085,KL: 237.5590
INFO:scDualGN.run:Epoch 9/100,Overall loss:40976.6774,MSE:2048.4386,MSE1:2.7177,

4096


INFO:scDualGN.run:obtain dual-VAE z-vector and x_bar
INFO:scDualGN.run:perform kmeans to cluster....
INFO:scDualGN.run:training scDualGN...
INFO:scDualGN.run:epoch 1/30
INFO:scDualGN.evalution:acc=0.7036, nmi=0.7824, ari=0.6461
INFO:scDualGN.run:Epoch 1/30, Loss - overall: 31008.4873,daul_VAE:31008.4600,KL:0.0137,Center:141.0299
INFO:scDualGN.run:Epoch 2/30, Loss - overall: 30965.8027,daul_VAE:30965.7754,KL:0.0140,Center:140.5327
INFO:scDualGN.run:Epoch 3/30, Loss - overall: 30180.7002,daul_VAE:30180.6729,KL:0.0138,Center:139.6896
INFO:scDualGN.run:Epoch 4/30, Loss - overall: 30444.5615,daul_VAE:30444.5342,KL:0.0135,Center:140.8080
INFO:scDualGN.run:Epoch 5/30, Loss - overall: 30375.7744,daul_VAE:30375.7471,KL:0.0140,Center:140.3284
INFO:scDualGN.run:epoch 6/30
INFO:scDualGN.evalution:acc=0.7095, nmi=0.7814, ari=0.6491
INFO:scDualGN.run:Epoch 6/30, Loss - overall: 30769.1426,daul_VAE:30769.1172,KL:0.0121,Center:141.2358
INFO:scDualGN.run:Epoch 7/30, Loss - overall: 29959.2646,daul_VAE:

running time = 42.69543981552124
